In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('Financial_Consumer_Complaints.csv')

In [4]:
df.head(3)

,Complaint ID,Date Sumbited,Product,Sub-product,Issue,Sub-issue,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date Received,Company response to consumer,Timely response?,Consumer disputed?
0,2824926,2/24/18,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,NaN,Cumulus Financial,FL,33179,NaN,Consent not provided,Web,2/24/18,Closed with explanation,Yes,NaN
1,1515991,8/12/15,Debt collection,Credit card,Improper contact or sharing of info,Contacted employer after asked not to,NaN,Cumulus Financial,CA,93561,NaN,Consent not provided,Web,8/12/15,Closed with explanation,Yes,No
2,1145261,12/5/14,Bank account or service,Checking account,Deposits and withdrawals,"""""",NaN,Cumulus Financial,CA,91792,NaN,NaN,Referral,12/8/14,Closed with explanation,Yes,No


# Q1. Which Product has the most of the complaints? State top five with Count

In [7]:
q1 = df.groupby('Product')['Complaint ID'].agg('count').sort_values(ascending = False)
q1.head(5)

Product
Credit card                    19176
Checking or savings account    13436
Mortgage                       12470
Credit card or prepaid card    10241
Bank account or service         9893
Name: Complaint ID, dtype: int64

# Q2. What kind of Issues Distinct Issues are present in Mortgage Product?

In [11]:
q2 = df[df['Product'] == 'Mortgage']['Issue'].unique()
q2

array(['Trouble during payment process', 'Struggling to pay mortgage',
       'Settlement process and costs', 'Closing on a mortgage',
       'Credit decision / Underwriting', 'Other',
       'Applying for a mortgage or refinancing an existing mortgage',
       'Improper use of your report',
       'Incorrect information on your report',
       "Problem with a credit reporting company's investigation into an existing problem",
       'Credit monitoring or identity theft protection services',
       'Unable to get your credit report or credit score',
       'Problem with fraud alerts or security freezes'], dtype=object)

# Q3. Which State has the most complaints for Credit Card? And how many were timely closed?

In [14]:
df.head(1)

,Complaint ID,Date Sumbited,Product,Sub-product,Issue,Sub-issue,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date Received,Company response to consumer,Timely response?,Consumer disputed?
0,2824926,2/24/18,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,NaN,Cumulus Financial,FL,33179,NaN,Consent not provided,Web,2/24/18,Closed with explanation,Yes,NaN


In [18]:
q3 = df[ df['Product'].str.lower().str.contains('credit card')]
q3_1 = q3.groupby('State')['Complaint ID'].agg('count').sort_values(ascending = False)
q3_1.head(1)

State
CA    4305
Name: Complaint ID, dtype: int64

In [19]:
q3_2 = q3[ q3['Timely response?'] == 'Yes' ].groupby('State')['Complaint ID'].agg('count').sort_values(ascending = False)
q3_2.head(1)

State
CA    4248
Name: Complaint ID, dtype: int64

# Q4. Summarize # of complaints received from sources (web, post) from state of New York NY only?

In [28]:
q4 = df[ (df['State'] == 'NY')].groupby('Submitted via')['Complaint ID'].agg('count').sort_values(ascending = False)
q4

Submitted via
Web            3251
Referral       2265
Phone           702
Postal mail     517
Fax             105
Email             1
Name: Complaint ID, dtype: int64

# Q5. What is the popular company response to complaints by number of complaints?

In [30]:
q5 = df.groupby('Company response to consumer')['Complaint ID'].agg('count').sort_values(ascending = False)
q5

Company response to consumer
Closed with explanation            54666
Closed with monetary relief        11693
Closed with non-monetary relief     5912
Closed without relief               1779
Closed with relief                   938
In progress                          283
Closed                               241
Untimely response                      1
Name: Complaint ID, dtype: int64

# Q6. How many complaints were timely resolved and provide breakup by both timely resolve & consumer dispute status?

In [35]:
q6 = df.groupby(['Timely response?', 'Consumer disputed?'], dropna=False)['Complaint ID'].agg('count').sort_values(ascending = False)
q6

Timely response?  Consumer disputed?
Yes               NaN                   36312
                  No                    30473
                  Yes                    7259
No                No                      730
                  NaN                     635
                  Yes                     104
Name: Complaint ID, dtype: int64

In [31]:
df.head(1)

,Complaint ID,Date Sumbited,Product,Sub-product,Issue,Sub-issue,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date Received,Company response to consumer,Timely response?,Consumer disputed?
0,2824926,2/24/18,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,NaN,Cumulus Financial,FL,33179,NaN,Consent not provided,Web,2/24/18,Closed with explanation,Yes,NaN


# Q7. Which is the most popular year for complaints related to “Vehicle loan or lease”?

In [42]:
q7_0 = df.copy()
q7_0['year'] = pd.DatetimeIndex(pd.to_datetime(q7_0['Date Sumbited'])).year
q7 = q7_0[q7_0['Product'] == 'Vehicle loan or lease'].groupby('year')['Complaint ID'].agg('count').sort_values(ascending= False)
q7

year
2018    351
2019    266
2020    245
2017    208
Name: Complaint ID, dtype: int64

# Q8. What is the average gap in number of days between complaints submitted and complaint received for the post mail?

In [95]:
q8_0 = df.copy()
q8_0['gap_in_days'] = pd.DatetimeIndex(pd.to_datetime(q8_0['Date Received'])).dayofyear  - pd.DatetimeIndex(pd.to_datetime(q8_0['Date Sumbited'])).dayofyear
q8_0 = q8_0[ q8_0['gap_in_days'] >= 0]
q8 = q8_0[ q8_0['Submitted via'].str.lower().str.contains('post')]['gap_in_days'].agg('mean')
q8

2.6002651013065705

In [96]:
q8_2 = q8_0.groupby('Submitted via')['gap_in_days'].agg('mean')
q8_2

Submitted via
Email          5.550000
Fax            2.584769
Phone          2.381676
Postal mail    2.600265
Referral       4.639481
Web            1.250414
Name: gap_in_days, dtype: float64

In [109]:
q8_3 = df.copy()
q8_3['gap_in_days'] = (pd.to_datetime(q8_0['Date Received']) - pd.to_datetime(q8_0['Date Sumbited'])).dt.days
q8_3 = q8_3[ q8_3['gap_in_days'] >= 0]
q8_4 = q8_3.groupby('Submitted via')['gap_in_days'].agg('mean')

q8_4

Submitted via
Email          5.550000
Fax            2.584769
Phone          2.508057
Postal mail    2.807802
Referral       4.656018
Web            1.260170
Name: gap_in_days, dtype: float64

# Q9. Which complaints have not been resolved till date. What is the minimum and maximum no. of days passed from submission date?